In [1]:
import pandas as pd
import pprint
import os

In [2]:
annotations = pd.read_csv("/Users/jedi/courses/im2im2im/dataset/celeba/list_attr_celeba_copy.txt", 
                          sep=('\s+'),
                          header=0) 
data_split = pd.read_csv("/Users/jedi/courses/im2im2im/dataset/celeba/list_eval_partition.txt", 
                          sep=('\s+'),
                          header=None,
                         names=['images', 'dataset']) 

In [3]:
combined_data = pd.merge(annotations, data_split, left_index=True, right_index=True, how='outer')
cols = combined_data.columns.get_values()

In [7]:
pprint.pprint(cols)

array(['Images', '5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive',
       'Bags_Under_Eyes', 'Bald', 'Bangs', 'Big_Lips', 'Big_Nose',
       'Black_Hair', 'Blond_Hair', 'Blurry', 'Brown_Hair',
       'Bushy_Eyebrows', 'Chubby', 'Double_Chin', 'Eyeglasses', 'Goatee',
       'Gray_Hair', 'Heavy_Makeup', 'High_Cheekbones', 'Male',
       'Mouth_Slightly_Open', 'Mustache', 'Narrow_Eyes', 'No_Beard',
       'Oval_Face', 'Pale_Skin', 'Pointy_Nose', 'Receding_Hairline',
       'Rosy_Cheeks', 'Sideburns', 'Smiling', 'Straight_Hair', 'Wavy_Hair',
       'Wearing_Earrings', 'Wearing_Hat', 'Wearing_Lipstick',
       'Wearing_Necklace', 'Wearing_Necktie', 'Young', 'images', 'dataset'], dtype=object)


In [4]:
# Datastats
print("Num datapoints for each characteristic")
attr_fd = []
for col in cols:
    attr_fd.append([col, len(combined_data[(combined_data[col]== 1)].index)])
    #print("{}: {}".format(col, num))
for i in sorted(attr_fd, key=lambda x:x[1], reverse=True):
    print(i)

Num datapoints for each characteristic
['No_Beard', 169158]
['Young', 156734]
['Attractive', 103833]
['Mouth_Slightly_Open', 97942]
['Smiling', 97669]
['Wearing_Lipstick', 95715]
['High_Cheekbones', 92189]
['Male', 84434]
['Heavy_Makeup', 78390]
['Wavy_Hair', 64744]
['Oval_Face', 57567]
['Pointy_Nose', 56210]
['Arched_Eyebrows', 54090]
['Big_Lips', 48785]
['Black_Hair', 48472]
['Big_Nose', 47516]
['Straight_Hair', 42222]
['Brown_Hair', 41572]
['Bags_Under_Eyes', 41446]
['Wearing_Earrings', 38276]
['Bangs', 30709]
['Blond_Hair', 29983]
['Bushy_Eyebrows', 28803]
['Wearing_Necklace', 24913]
['Narrow_Eyes', 23329]
['5_o_Clock_Shadow', 22516]
['dataset', 19867]
['Receding_Hairline', 16163]
['Wearing_Necktie', 14732]
['Rosy_Cheeks', 13315]
['Eyeglasses', 13193]
['Goatee', 12716]
['Chubby', 11663]
['Sideburns', 11449]
['Blurry', 10312]
['Wearing_Hat', 9818]
['Double_Chin', 9459]
['Pale_Skin', 8701]
['Gray_Hair', 8499]
['Mustache', 8417]
['Bald', 4547]
['Images', 0]
['images', 0]


In [10]:
# Make lists
blond_no_eye = combined_data[(combined_data['Eyeglasses']== -1) & 
                             (combined_data['Blond_Hair']== 1) & 
                             (combined_data['dataset'] == 0)]
brunette_no_eye = combined_data[(combined_data['Eyeglasses']== -1) & 
                             (combined_data['Brown_Hair']== 1) & 
                             (combined_data['dataset'] == 0)]
eye_noblond_no_brunette = combined_data[(combined_data['Eyeglasses']== 1) & 
                             (combined_data['Blond_Hair']== -1) & 
                             (combined_data['Brown_Hair']== -1) & 
                             (combined_data['dataset'] == 0)]
noeye_noblond_no_brunette = combined_data[(combined_data['Eyeglasses']== -1) & 
                             (combined_data['Blond_Hair']== -1) & 
                             (combined_data['Brown_Hair']== -1) & 
                             (combined_data['dataset'] == 0)]

In [8]:
print(len(goatee.index))
print(len(no_goatee.index))
# print(len(gray.index))
# print(len(bald.index))

10337
152433


In [4]:
# # Make lists
# eye_no_smiling = combined_data[(combined_data['Eyeglasses']== 1) & 
#                              (combined_data['Smiling']== -1) & 
#                              (combined_data['dataset'] == 0)]
# no_eye_no_smiling = combined_data[(combined_data['Eyeglasses']== -1) & 
#                              (combined_data['Smiling']== -1) & 
#                              (combined_data['dataset'] == 0)]
# no_eye_smiling = combined_data[(combined_data['Smiling']== 1) &
#                             (combined_data['Eyeglasses']== -1) & 
#                              (combined_data['dataset'] == 0)]
# no_smiling = combined_data[(combined_data['Smiling']== -1) & 
#                             (combined_data['Eyeglasses']== -1) &                            
#                              (combined_data['dataset'] == 0)]

# blond = combined_data[(combined_data['Blond_Hair']== 1) & 
#                              (combined_data['dataset'] == 0)]
# black = combined_data[(combined_data['Black_Hair']== 1) & 
#                              (combined_data['dataset'] == 0)]
# gray = combined_data[(combined_data['Gray_Hair']== 1) &
#                              (combined_data['dataset'] == 0)]
# bald = combined_data[(combined_data['Bald']== 1) & 
#                              (combined_data['dataset'] == 0)]
# goatee = combined_data[(combined_data['Chubby']== 1) &
#                              (combined_data['dataset'] == 0)]
# no_goatee = combined_data[(combined_data['Wearing_Lipstick']== -1) & 
#                              (combined_data['dataset'] == 0)]

class0 = combined_data[(combined_data['Smiling']== -1) & 
                             (combined_data['Blond_Hair']== 1) & 
                             (combined_data['dataset'] == 0)]
class1 = combined_data[(combined_data['Smiling']== -1) & 
                             (combined_data['Brown_Hair']== 1) & 
                             (combined_data['dataset'] == 0)]
class2 = combined_data[(combined_data['Smiling']== 1) & 
                             (combined_data['Blond_Hair']== -1) &
                             (combined_data['Brown_Hair']== -1) &                                         
                             (combined_data['dataset'] == 0)]
class3 = combined_data[(combined_data['Smiling']== 1) & 
                             (combined_data['Blond_Hair']== -1) &
                             (combined_data['Brown_Hair']== 1) &                                         
                             (combined_data['dataset'] == 0)]

class0 = combined_data[(combined_data['Smiling']== -1) & 
                             (combined_data['Eyeglasses']== 1) & 
                             (combined_data['dataset'] == 0)]
class1 = combined_data[(combined_data['Smiling']== -1) & 
                             (combined_data['Eyeglasses']== -1) & 
                             (combined_data['dataset'] == 0)]
class2 = combined_data[(combined_data['Smiling']== 1) & 
                             (combined_data['Eyeglasses']== -1) &
                             (combined_data['dataset'] == 0)]


In [5]:
print(len(class0.index))
print(len(class1.index))
print(len(class2.index))
print(len(class3.index))


6275
78415
73834
16044


In [6]:
alldata = []
for i in class0.values:
    alldata.append([i[0], 0])
for i in class1.values:
    alldata.append([i[0], 1])
for i in class2.values:
    alldata.append([i[0], 2])
# for i in class3.values:
#     alldata.append([i[0], 3])
        
import random        
random.shuffle(alldata)
part = int(len(alldata)*0.8)
traindata = alldata[:part]
valdata = alldata[part:]    
f = open("/Users/jedi/courses/im2im2im/nvidia_original/datasets/celeba/lists/anant_im2im2im_train_2list.txt", "w")
for i in traindata:
    print>>f, i[0]+","+str(i[1])
f.close()    
f = open("/Users/jedi/courses/im2im2im/nvidia_original/datasets/celeba/lists/anant_im2im2im_val_2list.txt", "w")
for i in valdata:
    print>>f, i[0]+","+str(i[1])
f.close()    

In [15]:

alldata = [[i[0], 0] for i in blond.values]+[[i[0], 1] for i in black.values]+[[i[0], 2] for i in gray.values]+[[i[0], 3] for i in bald.values]
random.shuffle(alldata)
part = int(len(alldata)*0.8)
traindata = alldata[:part]
valdata = alldata[part:]    
f = open("/Users/jedi/courses/im2im2im/nvidia_original/datasets/celeba/lists/anant_hairs_train.txt", "w")
for i in traindata:
    print>>f, i[0]+","+str(i[1])
f.close()    
f = open("a/Users/jedi/courses/im2im2im/nvidia_original/datasets/celeba/lists/anant_hairs_val.txt", "w")
for i in valdata:
    print>>f, i[0]+","+str(i[1])
f.close()    

In [8]:
path = "/Users/jedi/courses/im2im2im/nvidia_original/datasets/celeba/lists"
# writefile(path, 'anant_male.txt', male, 1000000, 6)
# writefile(path, 'anant_female.txt', female, 10000, 7)

writefile(path, 'anant_blond_no_smiling.txt', class0, 1000000, 0)
writefile(path, 'anant_bruntte_no_smiling.txt', class1, 1000000, 1)
writefile(path, 'anant_smiling_no_blondbrunette.txt', class2, 1000000, 2)
writefile(path, 'anant_no_smiling_no_blondbrunette.txt', class3, 1000000, 3)

Data written to /Users/jedi/courses/im2im2im/nvidia_original/datasets/celeba/lists/anant_blond_no_smiling.txt
Data written to /Users/jedi/courses/im2im2im/nvidia_original/datasets/celeba/lists/anant_bruntte_no_smiling.txt
Data written to /Users/jedi/courses/im2im2im/nvidia_original/datasets/celeba/lists/anant_smiling_no_blondbrunette.txt
Data written to /Users/jedi/courses/im2im2im/nvidia_original/datasets/celeba/lists/anant_no_smiling_no_blondbrunette.txt


In [10]:
from sklearn.utils import shuffle
path = "/Users/jedi/courses/im2im2im/nvidia_original/datasets/celeba/lists"
for i, _at in enumerate(attrs):
    shuffle(distributions[_at])
    part = int(len(distributions[_at])*0.8)
    traindata = distributions[_at][:part]
    valdata = distributions[_at][part:]    
    writefile(path, "anant_%s_train.txt"%_at, traindata, 1000000, i)
    writefile(path, "anant_%s_val.txt"%_at, valdata, 1000000, i)    

Data written to /Users/jedi/courses/im2im2im/nvidia_original/datasets/celeba/lists/anant_Smiling_train.txt
Data written to /Users/jedi/courses/im2im2im/nvidia_original/datasets/celeba/lists/anant_Smiling_val.txt
Data written to /Users/jedi/courses/im2im2im/nvidia_original/datasets/celeba/lists/anant_Eyeglasses_train.txt
Data written to /Users/jedi/courses/im2im2im/nvidia_original/datasets/celeba/lists/anant_Eyeglasses_val.txt
Data written to /Users/jedi/courses/im2im2im/nvidia_original/datasets/celeba/lists/anant_Blond_Hair_train.txt
Data written to /Users/jedi/courses/im2im2im/nvidia_original/datasets/celeba/lists/anant_Blond_Hair_val.txt
Data written to /Users/jedi/courses/im2im2im/nvidia_original/datasets/celeba/lists/anant_Black_Hair_train.txt
Data written to /Users/jedi/courses/im2im2im/nvidia_original/datasets/celeba/lists/anant_Black_Hair_val.txt


In [6]:
def writefile(path, filename, df, num_egs, label=0):
    with open(os.path.join(path, filename), 'w') as f:
        for i, (idx, row) in enumerate(df.iterrows()):
            line = str(row['Images']) + " %d\n"%label
            f.write(line)
            if i >= num_egs:
                break
    print("Data written to {}/{}".format(path, filename))

In [7]:
# Write out lists of first 9200 examples
path = '/Users/lauragraesser/Documents/NYU_Courses/CV/im2im2im/nvidia_original/datasets/celeba/lists'
# filename = 'blond_no_eye.txt'
# writefile(path, filename, blond_no_eye, 9200)
# filename = 'brunette_no_eye.txt'
# writefile(path, filename, brunette_no_eye, 9200)
# filename = 'eye_noblond_no_brunette.txt'
# writefile(path, filename, eye_noblond_no_brunette, 9200)
# filename = 'noeye_noblond_no_brunette.txt'
# writefile(path, filename, noeye_noblond_no_brunette, 9200)

In [18]:
# smiling_noblond_no_brunette = combined_data[(combined_data['Smiling']== 1) & 
#                              (combined_data['Blond_Hair']== -1) & 
#                              (combined_data['Brown_Hair']== -1) & 
#                              (combined_data['dataset'] == 0)]
# nosmiling_noblond_no_brunette = combined_data[(combined_data['Smiling']== -1) & 
#                              (combined_data['Blond_Hair']== -1) & 
#                              (combined_data['Brown_Hair']== -1) & 
#                              (combined_data['dataset'] == 0)]
male = combined_data[(combined_data['Male']== 1) &  
                             (combined_data['dataset'] == 0)]
female = combined_data[(combined_data['Male']== -1) &  
                             (combined_data['dataset'] == 0)]

In [20]:
# print(len(smiling_noblond_no_brunette.index))
# print(len(nosmiling_noblond_no_brunette.index))
print(len(male.index))
print(len(female.index))

68261
94509


In [12]:
filename = 'blond_no_eye_20k.txt'
writefile(path, filename, blond_no_eye, 20000)
filename = 'brunette_no_eye_20k.txt'
writefile(path, filename, brunette_no_eye, 20000)

Data written to /Users/lauragraesser/Documents/NYU_Courses/CV/im2im2im/nvidia_original/datasets/celeba/lists/blond_no_eye_20k.txt
Data written to /Users/lauragraesser/Documents/NYU_Courses/CV/im2im2im/nvidia_original/datasets/celeba/lists/brunette_no_eye_20k.txt


In [13]:
filename = 'smiling_noblond_no_brunette_20k.txt'
writefile(path, filename, smiling_noblond_no_brunette, 20000)
filename = 'nosmiling_noblond_no_brunette_20k.txt'
writefile(path, filename, nosmiling_noblond_no_brunette, 20000)

Data written to /Users/lauragraesser/Documents/NYU_Courses/CV/im2im2im/nvidia_original/datasets/celeba/lists/smiling_noblond_no_brunette_20k.txt
Data written to /Users/lauragraesser/Documents/NYU_Courses/CV/im2im2im/nvidia_original/datasets/celeba/lists/nosmiling_noblond_no_brunette_20k.txt


In [14]:
filename = 'male_20k.txt'
writefile(path, filename, male, 20000)
filename = 'female_20k.txt'
writefile(path, filename, female, 20000)

Data written to /Users/lauragraesser/Documents/NYU_Courses/CV/im2im2im/nvidia_original/datasets/celeba/lists/male_20k.txt
Data written to /Users/lauragraesser/Documents/NYU_Courses/CV/im2im2im/nvidia_original/datasets/celeba/lists/female_20k.txt


In [8]:
hat = combined_data[(combined_data['Wearing_Hat']== 1) &  
                             (combined_data['dataset'] == 0)]
nohat = combined_data[(combined_data['Wearing_Hat']== -1) &  
                             (combined_data['dataset'] == 0)]
necktie = combined_data[(combined_data['Wearing_Necktie']== 1) &  
                             (combined_data['dataset'] == 0)]
nonecktie = combined_data[(combined_data['Wearing_Necktie']== -1) &  
                             (combined_data['dataset'] == 0)]

In [9]:
print(len(hat.index))
print(len(nohat.index))
print(len(necktie.index))
print(len(nonecktie.index))

8039
154731
11890
150880


In [10]:
filename = 'hat_8k.txt'
writefile(path, filename, hat, 8000)
filename = 'nohat_8k.txt'
writefile(path, filename, nohat, 8000)
filename = 'necktie_11k.txt'
writefile(path, filename, necktie, 11000)
filename = 'nonecktie_11k.txt'
writefile(path, filename, nonecktie, 11000)

Data written to /Users/lauragraesser/Documents/NYU_Courses/CV/im2im2im/nvidia_original/datasets/celeba/lists/hat_8k.txt
Data written to /Users/lauragraesser/Documents/NYU_Courses/CV/im2im2im/nvidia_original/datasets/celeba/lists/nohat_8k.txt
Data written to /Users/lauragraesser/Documents/NYU_Courses/CV/im2im2im/nvidia_original/datasets/celeba/lists/necktie_11k.txt
Data written to /Users/lauragraesser/Documents/NYU_Courses/CV/im2im2im/nvidia_original/datasets/celeba/lists/nonecktie_11k.txt


In [10]:
eyeglasses = combined_data[(combined_data['Eyeglasses']== 1) &  
                             (combined_data['dataset'] == 0)]
noeyeglasses = combined_data[(combined_data['Eyeglasses']== -1) &  
                             (combined_data['dataset'] == 0)]

In [11]:
print(len(eyeglasses.index))
print(len(noeyeglasses.index))

10521
152249


In [12]:
filename = 'eyeglasses_10k.txt'
writefile(path, filename, eyeglasses, 10000)
filename = 'noneyeglasses_10k.txt'
writefile(path, filename, noeyeglasses, 10000)

Data written to /Users/lauragraesser/Documents/NYU_Courses/CV/im2im2im/nvidia_original/datasets/celeba/lists/eyeglasses_10k.txt
Data written to /Users/lauragraesser/Documents/NYU_Courses/CV/im2im2im/nvidia_original/datasets/celeba/lists/noneyeglasses_10k.txt
